### Feature Union in sklearn

FeatureUnion combines several transformer objects into a new transformer that combines their output. A FeatureUnion takes a list of transformer objects. During fitting, each of these is fit to the data independently. For transforming data, the transformers are applied in parallel, and the sample vectors they output are concatenated end-to-end into larger vectors.

+ FeatureUnion concatenates the results of multiple transformer objects.
+ It applies a list of transformer objects in parallel, and their outputs are concatenated side-by-side into a larger matrix.

Here's a very simple and basic example usage:

In [42]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures

# Load the iris dataset
iris = load_iris(as_frame=True)
X, y = iris.data, iris.target

In [25]:
X

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [45]:
# pipeline 1 applies standard scaler to the data and selects the top 2 components using PCA
pipe1 = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=2))
])


# polynomial features to get 2nd degree interaction features
polyfeatures = PolynomialFeatures(degree=2)

# combine these 2 transformations using a feature union
# the end result of both these transformations will be concatenated in a single matrix
combined_features = FeatureUnion([
    ("pipe1", pipe1),
    ("polynomial_features", polyfeatures)
]).set_output(transform="pandas")

combined_features

FeatureUnion(transformer_list=[('pipe1',
                                Pipeline(steps=[('scaler', StandardScaler()),
                                                ('pca', PCA(n_components=2))])),
                               ('polynomial_features', PolynomialFeatures())])

In [44]:
# Use combined features to transform dataset
X_features = combined_features.fit_transform(X, y)
X_features

,pipe1__pca0,pipe1__pca1,polynomial_features__1,polynomial_features__sepal length (cm),polynomial_features__sepal width (cm),polynomial_features__petal length (cm),polynomial_features__petal width (cm),polynomial_features__sepal length (cm)^2,polynomial_features__sepal length (cm) sepal width (cm),polynomial_features__sepal length (cm) petal length (cm),polynomial_features__sepal length (cm) petal width (cm),polynomial_features__sepal width (cm)^2,polynomial_features__sepal width (cm) petal length (cm),polynomial_features__sepal width (cm) petal width (cm),polynomial_features__petal length (cm)^2,polynomial_features__petal length (cm) petal width (cm),polynomial_features__petal width (cm)^2
0,-2.264703,0.480027,1.0,5.1,3.5,1.4,0.2,26.01,17.85,7.14,1.02,12.25,4.90,0.70,1.96,0.28,0.04
1,-2.080961,-0.674134,1.0,4.9,3.0,1.4,0.2,24.01,14.70,6.86,0.98,9.00,4.20,0.60,1.96,0.28,0.04
2,-2.364229,-0.341908,1.0,4.7,3.2,1.3,0.2,22.09,15.04,6.11,0.94,10.24,4.16,0.64,1.69,0.26,0.04
3,-2.299384,-0.597395,1.0,4.6,3.1,1.5,0.2,21.16,14.26,6.90,0.92,9.61,4.65,0.62,2.25,0.30,0.04
4,-2.389842,0.646835,1.0,5.0,3.6,1.4,0.2,25.00,18.00,7.00,1.00,12.96,5.04,0.72,1.96,0.28,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,1.870503,0.386966,1.0,6.7,3.0,5.2,2.3,44.89,20.10,34.84,15.41,9.00,15.60,6.90,27.04,11.96,5.29
146,1.564580,-0.896687,1.0,6.3,2.5,5.0,1.9,39.69,15.75,31.50,11.97,6.25,12.50,4.75,25.00,9.50,3.61
147,1.521170,0.269069,1.0,6.5,3.0,5.2,2.0,42.25,19.50,33.80,13.00,9.00,15.60,6.00,27.04,10.40,4.00
148,1.372788,1.011254,1.0,6.2,3.4,5.4,2.3,38.44,21.08,33.48,14.26,11.56,18.36,7.82,29.16,12.42,5.29


In [41]:
# Use a classifier on the combined features
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_features, y)

LogisticRegression(max_iter=1000)

In this example, we first load the iris dataset. We then create a pipeline that applies a `StandardScaler` and then extracts the top 2 features using `PCA`. We then create a polynomial features transformer to get degree 2 interaction features.

We then create a `FeatureUnion` that combines the pipeline(that applied Scaling and PCA) and polynomial feature transformations.

The `FeatureUnion` is fit to the data, which means it applies each transformation (the pipeline and the polynomial features) in parallel to the data, and then concatenates the results. This transformed data is then used to fit a `LogisticRegression` classifier.